In [ ]:
#importing libraries
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# csvs into variables

train_set = pd.read_csv("/kaggle/input/titanic/train.csv")
test_set = pd.read_csv("/kaggle/input/titanic/test.csv")
gender_submission_set = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

train_set.head()

In [ ]:
test_set.head()

In [ ]:
gender_submission_set.head()

In [ ]:
# In my opinion those fields will not help in model

train_set = train_set.drop(['Ticket','Cabin','Name'],axis=1)

test_set = test_set.drop(['Ticket','Cabin','Name'],axis=1)

In [ ]:
# I am changing categorical data to numerical
train_set = train_set.replace('male','1')
train_set = train_set.replace('female','0')

train_set = train_set.replace('S','0')
train_set = train_set.replace('C','1')
train_set = train_set.replace('Q','2')

test_set = test_set.replace('male','1')
test_set = test_set.replace('female','0')

test_set = test_set.replace('S','0')
test_set = test_set.replace('C','1')
test_set = test_set.replace('Q','2')

In [ ]:
train_set.head()

In [ ]:
test_set.head()

In [ ]:
# this is just a tmp variable
X_begin = train_set.drop('Survived',axis=1)

In [ ]:
# How much NaN do we have?
X_begin.isna().sum().sum()

In [ ]:
test_set.isna().sum().sum()

In [ ]:
# Getting rid of every NaN value
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
X = pd.DataFrame(my_imputer.fit_transform(X_begin))

X.columns=X_begin.columns
X.index=X_begin.index

test_set_final = pd.DataFrame(my_imputer.fit_transform(test_set))

test_set_final.columns=test_set.columns
test_set_final.index=test_set.index

In [ ]:
X.isna().sum().sum()

In [ ]:
test_set_final.isna().sum().sum()

In [ ]:
X

In [ ]:
test_set_final

In [ ]:
y= train_set['Survived']

In [ ]:
y

In [ ]:
# splitting train set for some more - to get validation how well every algorithm works
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

ss_test = StandardScaler()
test_set_final = ss_test.fit_transform(test_set_final)

In [ ]:
# Visit this webpage: https://www.learndatasci.com/glossary/binary-classification/
## training the models
models = {}

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression()

# # Support Vector Machines
# from sklearn.svm import LinearSVC
# models['Support Vector Machines'] = LinearSVC()

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
models['Naive Bayes'] = GaussianNB()

# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
models['K-Nearest Neighbor'] = KNeighborsClassifier()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy, precision, recall = {}, {}, {}

for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train, y_train)
    
    # Make predictions
    predictions = models[key].predict(X_test)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)

In [ ]:
df_model = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = accuracy.values()
df_model['Precision'] = precision.values()
df_model['Recall'] = recall.values()

df_model

In [ ]:
# I choose the best model
predictions_final = models['Random Forest'].predict(test_set_final)

In [ ]:
output = pd.DataFrame({'PassengerId': test_set.PassengerId, 'Survived': predictions_final})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
output